# <center> Линейная алгебра в контексте линейных методов. Практика 2

## <center> Прогнозирование выработки газа на скважинах. Часть 2

## Полиномиальная регрессия и регуляризация

Мы продолжаем работать над задачей от владельца компании «Газ-Таз-Ваз-Нефть» Василия.

Ранее мы построили модель линейной регрессии, которая прогнозирует выработку газа на скважине. Для этого мы с помощью матрицы корреляций и рассуждений отобрали некоррелированные, значимые для предсказания признаки. **Далее мы будем использовать именно их (см. [Практика 1, задание 1.5](../../hw_1/base/hw_1.ipynb))**

Мы хотим улучшить результат — уменьшить ошибку прогноза. Для этого мы воспользуемся моделью полиномиальной регрессии третьей степени. Однако теперь мы знаем, что полиномиальным моделям очень легко переобучиться под исходную выборку. Так как данных у нас не так много (всего 200 скважин), то для контроля качества модели мы будем использовать кросс-валидацию. 

Приступим! Выполните задания 2.1–2.5:


In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate

In [ ]:
df_cleaned.head(1)

,Por,AI,Brittle,VR,Prod
0,12.08,2.8,81.4,2.31,4165.196191


### Задание 2.1. (1 балл)

Стандаризируйте признаки с помощью `StandartScaler` из библиотеки `sklearn`. 

Затем сгенерируйте полиномиальные признаки третьего порядка на факторах, которые вы выбрали для обучения моделей. Для этого воспользуйтесь генератором полиномов `PolynomialFeatures` из библиотеки `sklearn`. Параметр `include_bias` установите в значение `False`.

Выведите на экран, сколько факторов у вас получилось после генерации полиномиальных признаков.

**Важно:** стандартизацию необходимо произвести до генерации полиномиальных факторов!

Обучите модель линейной регрессии из библиотеки `sklearn` (`LinearRegression`) на полученных полиномиальных факторах.

Используя кросс-валидацию оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * на основе отобранных факторов сгенерированы полиномиальные признаки третьего порядка;
    * построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).
- Задание выполнено неверно, не учтено одно или несколько из условий (**0 баллов**).

#### Решение

In [ ]:
df_cleaned.shape

(200, 5)

In [ ]:
X_updated.shape

(200, 4)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_updated)  # Стандартизированные данные

In [ ]:
poly = PolynomialFeatures(degree=3, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

In [ ]:
num_features = X_poly.shape[1]
print("Количество факторов после генерации полиномиальных признаков:", num_features)

Количество факторов после генерации полиномиальных признаков: 34


In [ ]:
model = LinearRegression()

scoring = {
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

In [ ]:
cv_results = cross_validate(model, X_poly, y_updated, cv=5, scoring=scoring, return_train_score=True)

In [ ]:
train_mse = abs(np.mean(cv_results['train_mse']))
train_mae = abs(np.mean(cv_results['train_mae']))
val_mse = abs(np.mean(cv_results['test_mse']))
val_mae = abs(np.mean(cv_results['test_mae']))

In [ ]:
print("Средняя MSE на тренировочных фолдах:", train_mse)
print("Средняя MAE на тренировочных фолдах:", train_mae)
print("Средняя MSE на валидационных фолдах:", val_mse)
print("Средняя MAE на валидационных фолдах:", val_mae)

Средняя MSE на тренировочных фолдах: 10162.07411833486
Средняя MAE на тренировочных фолдах: 75.34530982164883
Средняя MSE на валидационных фолдах: 24335.08926494503
Средняя MAE на валидационных фолдах: 110.47360945494015


Модель показала хорошие результаты на тренировочных данных, с низкими MSE и MAE, но на валидации всё стало хуже. Ошибки на валидации значительно выше, судя по всему - переобучение. То есть модель слишком "запомнила" обучающие данные и не может хорошо работать на новых данных. Это, скорее всего, из-за того, что добавление полиномиальных признаков сделало модель слишком сложной. 

Чтобы улучшить ситуацию, стоит попробовать добавить регуляризацию. На семинарах мы говорили, что регуляризация поможет модели лучше обобщать не зависеть от конкретных данных.

### Задание 2.2. (2 балла)

Теперь попробуем воспользоваться линейной регрессией с регуляризацией. Для начала возьмём $L_1$-регуляризацию.

Обучите модель `Lasso` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициент регуляризации (`alpha`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию, оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (Lasso), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

#### Решение

Чтобы найти наилучший коэффициент регуляризации `alpha`, применим метод кросс-валидации для подбора гиперпараметра.

In [ ]:
alpha_range = np.logspace(-4, 0, 50) 

In [ ]:
# я увеличил итерации до 10 k для надёжности
lasso = Lasso(max_iter=10_000)

In [ ]:
param_grid = {'alpha': alpha_range}
scoring = {
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

In [ ]:
grid_search = GridSearchCV(lasso, param_grid, cv=5, scoring=scoring, refit='mse')
grid_search.fit(X_poly, y_updated)

GridSearchCV(cv=5, estimator=Lasso(max_iter=10000),
             param_grid={'alpha': array([1.00000000e-04, 1.20679264e-04, 1.45634848e-04, 1.75751062e-04,
       2.12095089e-04, 2.55954792e-04, 3.08884360e-04, 3.72759372e-04,
       4.49843267e-04, 5.42867544e-04, 6.55128557e-04, 7.90604321e-04,
       9.54095476e-04, 1.15139540e-03, 1.38949549e-03, 1.67683294e-03,
       2.02358965e-03, 2.44205309e-03, 2.9...
       1.84206997e-01, 2.22299648e-01, 2.68269580e-01, 3.23745754e-01,
       3.90693994e-01, 4.71486636e-01, 5.68986603e-01, 6.86648845e-01,
       8.28642773e-01, 1.00000000e+00])},
             refit='mse',
             scoring={'mae': make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict'),
                      'mse': make_scorer(mean_squared_error, greater_is_better=False, response_method='predict')})

In [ ]:
best_alpha = grid_search.best_params_['alpha']
best_alpha

np.float64(1.0)

In [ ]:
lasso_best = Lasso(alpha=best_alpha, max_iter=10_000)
cv_results = cross_validate(lasso_best, X_poly, y_updated, cv=5, scoring=scoring, return_train_score=True)

In [ ]:
train_mse = -np.mean(cv_results['train_mse'])
train_mae = -np.mean(cv_results['train_mae'])
val_mse = -np.mean(cv_results['test_mse'])
val_mae = -np.mean(cv_results['test_mae'])

print("Лучшее значение alpha:", best_alpha)
print("Средняя MSE на тренировочных фолдах:", train_mse)
print("Средняя MAE на тренировочных фолдах:", train_mae)
print("Средняя MSE на валидационных фолдах:", val_mse)
print("Средняя MAE на валидационных фолдах:", val_mae)

Лучшее значение alpha: 1.0
Средняя MSE на тренировочных фолдах: 10403.136436685749
Средняя MAE на тренировочных фолдах: 75.72331479922748
Средняя MSE на валидационных фолдах: 20985.868882305193
Средняя MAE на валидационных фолдах: 102.87022400269976


Ну кстати, на валидации метрики улучшились! Средняя MSE на валидации уменьшилась с 24335 до 20985, а MAE снизилась со 110 до 102. Моя модель с регуляризацией стала лучше справляться с новыми данными. Приятно 😊

Но MSE всё равно очень высокий, почти 21k! 🤔 Это говорит о том, что местами выбросы модели критические

Но! Из очень приятного: у нас уже результаты лучше, чем в пунктах 5.4 и 5.5 🎉

Значит, мы движемся в правильном направлении)

### Задание 2.3. (2 балла)

Проделаем то же самое с $L_2$-регуляризацией.

Обучите модель `Ridge` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициент регуляризации (`alpha`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию оцените среднее значение выбранной вами метрики (или метрик) на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (Ridge), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

#### Решение

In [ ]:
alpha_range = np.logspace(-4, 1, 50)

In [ ]:
# снова возьмём 10к итераций, теперь на модели Ridge
ridge = Ridge(max_iter=10_000)

In [ ]:
param_grid = {'alpha': alpha_range}
scoring = {
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False)
}

In [ ]:
grid_search = GridSearchCV(ridge, param_grid, cv=5, scoring=scoring, refit='mse')
grid_search.fit(X_poly, y_updated)

GridSearchCV(cv=5, estimator=Ridge(max_iter=10000),
             param_grid={'alpha': array([1.00000000e-04, 1.26485522e-04, 1.59985872e-04, 2.02358965e-04,
       2.55954792e-04, 3.23745754e-04, 4.09491506e-04, 5.17947468e-04,
       6.55128557e-04, 8.28642773e-04, 1.04811313e-03, 1.32571137e-03,
       1.67683294e-03, 2.12095089e-03, 2.68269580e-03, 3.39322177e-03,
       4.29193426e-03, 5.42867544e-03, 6.8...
       1.20679264e+00, 1.52641797e+00, 1.93069773e+00, 2.44205309e+00,
       3.08884360e+00, 3.90693994e+00, 4.94171336e+00, 6.25055193e+00,
       7.90604321e+00, 1.00000000e+01])},
             refit='mse',
             scoring={'mae': make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict'),
                      'mse': make_scorer(mean_squared_error, greater_is_better=False, response_method='predict')})

In [ ]:
best_alpha = grid_search.best_params_['alpha']
best_alpha

np.float64(0.5963623316594643)

Опа, у нас рассчитался новый альфа

In [ ]:
ridge_best = Ridge(alpha=best_alpha, max_iter=10_000)
cv_results = cross_validate(ridge_best, X_poly, y_updated, cv=5, scoring=scoring, return_train_score=True)

In [ ]:
train_mse = -np.mean(cv_results['train_mse'])
train_mae = -np.mean(cv_results['train_mae'])
val_mse = -np.mean(cv_results['test_mse'])
val_mae = -np.mean(cv_results['test_mae'])

print("Лучшее значение alpha:", best_alpha)
print("Средняя MSE на тренировочных фолдах:", train_mse)
print("Средняя MAE на тренировочных фолдах:", train_mae)
print("Средняя MSE на валидационных фолдах:", val_mse)
print("Средняя MAE на валидационных фолдах:", val_mae)

Лучшее значение alpha: 0.5963623316594643
Средняя MSE на тренировочных фолдах: 10309.479907216828
Средняя MAE на тренировочных фолдах: 76.08630797193186
Средняя MSE на валидационных фолдах: 23807.00621678122
Средняя MAE на валидационных фолдах: 110.55805160995905


Результаты у $L_2$ чуть похуже чем у $L_1$. Причём на валидации проседают и `MSE`, и `MAE` - то есть полный пролёт. Немного, но хуже. Значит, при прочих равных мы бы выбрали $L_1$, если бы могли выбрать только одну модель

### Задание 2.4. (2 балла)

Наконец, настало время комбинировать $L_1$ и $L_2$ -регуляризации.

Обучите модель `ElasticNet` из библиотеки `sklearn` на полученных полиномиальных факторах, предварительно стандартизировав факторы. 

Коэффициенты регуляризации (`alpha` и `l1-ratio`) подберите самостоятельно с помощью любого известного вам метода подбора гиперпаметров.

Используя кросс-валидацию, оцените среднее значение метрики MAPE на тренировочных и валидационных фолдах.

Проинтерпретируйте полученные результаты.

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**2 балла**): 
    * правильно построена модель полиномиальной регрессии (линейной регрессии на полиномиальных признаках) с регуляризацией (ElasticNet), учтены условия необходимости масштабирования факторов для построения модели;
    * приведён код для подбора параметра регуляризации (вручную или с помощью библиотечных инструментов);
    * с помощью кросс-валидации оценено среднее значение выбранной студентом метрики (или метрик) на тренировочных и валидационных фолдах (метрика должна быть выбрана корректно).

- Задание выполнено верно, но не учтено одно из условий (**1 балл**).
- Задание выполнено неверно, не учтено несколько условий (**0 баллов**).

#### Решение

In [ ]:
alpha_range = np.logspace(-4, 1, 50)  
l1_ratio_range = np.linspace(0, 1, 10) 

# для эластика также возьмём 10 к итераций
elastic_net = ElasticNet(max_iter=10_000)

In [ ]:
param_grid = {'alpha': alpha_range, 'l1_ratio': l1_ratio_range}

scoring = {
    'mse': make_scorer(mean_squared_error, greater_is_better=False),
    'mae': make_scorer(mean_absolute_error, greater_is_better=False),
    'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False)
}

In [ ]:
grid_search = GridSearchCV(elastic_net, param_grid, cv=5, scoring=scoring, refit='mse')
grid_search.fit(X_poly, y_updated)

C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.763e+05, tolerance: 1.612e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.515e+05, tolerance: 1.545e+04 Linear regression models with null weight for the l1 regularization term are more efficiently fi

GridSearchCV(cv=5, estimator=ElasticNet(max_iter=10000),
             param_grid={'alpha': array([1.00000000e-04, 1.26485522e-04, 1.59985872e-04, 2.02358965e-04,
       2.55954792e-04, 3.23745754e-04, 4.09491506e-04, 5.17947468e-04,
       6.55128557e-04, 8.28642773e-04, 1.04811313e-03, 1.32571137e-03,
       1.67683294e-03, 2.12095089e-03, 2.68269580e-03, 3.39322177e-03,
       4.29193426e-03, 5.42867544e-0...
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])},
             refit='mse',
             scoring={'mae': make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict'),
                      'mape': make_scorer(mean_absolute_percentage_error, greater_is_better=False, response_method='predict'),
                      'mse': make_scorer(mean_squared_error, greater_is_better=False, response_method='predict')})

У нас вылезает `ConvergenceWarning` 😔 

Я погуглил и потестил разные варианты `max_iter`, `tol` - проблема остаётся. Похоже, косяк где-то на уровне метода.

Проверим, какие будут метрики несмотря на предупреждения

In [ ]:
best_alpha = grid_search.best_params_['alpha']
best_l1_ratio = grid_search.best_params_['l1_ratio']

elastic_net_best = ElasticNet(alpha=best_alpha, l1_ratio=best_l1_ratio, max_iter=10000)

In [ ]:
cv_results = cross_validate(elastic_net_best, X_poly, y_updated, cv=5, scoring=scoring, return_train_score=True)

In [ ]:
train_mse = -np.mean(cv_results['train_mse'])
train_mae = -np.mean(cv_results['train_mae'])
train_mape = -np.mean(cv_results['train_mape'])
val_mse = -np.mean(cv_results['test_mse'])
val_mae = -np.mean(cv_results['test_mae'])
val_mape = -np.mean(cv_results['test_mape'])

print("Лучшее значение alpha:", best_alpha)
print("Лучшее значение l1_ratio:", best_l1_ratio)
print("Средняя MSE на тренировочных фолдах:", train_mse)
print("Средняя MAE на тренировочных фолдах:", train_mae)
print("Средняя MAPE на тренировочных фолдах:", train_mape)
print("Средняя MSE на валидационных фолдах:", val_mse)
print("Средняя MAE на валидационных фолдах:", val_mae)
print("Средняя MAPE на валидационных фолдах:", val_mape)

Лучшее значение alpha: 6.250551925273976
Лучшее значение l1_ratio: 1.0
Средняя MSE на тренировочных фолдах: 11775.285354175494
Средняя MAE на тренировочных фолдах: 79.40722597494127
Средняя MAPE на тренировочных фолдах: 0.018548258982966186
Средняя MSE на валидационных фолдах: 17179.935747374115
Средняя MAE на валидационных фолдах: 95.99598853838488
Средняя MAPE на валидационных фолдах: 0.022817604619807696


Мы добились лучших показателей `MSE` и `MAE` - ошибок! 

У нас всё ещё есть эффект переобучения, поскольку ошибки на тренировочных датасетах меньше чем на валидации, но показатели на валидации отличные. Они гораздо лучше показателей в пунктах 5, и оптимальнее всех предыдущих показателей в пунктах 8. 

Супер! 😇

### Задание 2.5. (1 балл)

Завершением вашей работы будет сводная таблица результатов, которую вы представите на презентации заказчику Василию.

Составьте таблицу (DataFrame) со следующими столбцами (имена столбцов выберите самостоятельно):
* Наименование модели.
* Гиперпараметры (коэффициенты регуляризации, если таковые имеются), если нет — оставьте ячейку пустой.
* Использовались ли полиномиальные признаки при построении модели (Да/Нет или True/False).
* Значение выбранной метрики на тренировочных фолдах при кросс-валидации модели.
* Значение выбранной метрики на валидационных фолдах при кросс-валидации модели.

Сделайте финальный вывод по проделанной работе. Какая линейная модель из тех, что мы рассмотрели, показала наилучший результат с точки зрения качества на валидационных фолдах, а также с точки зрения переобученности?

**Критерии оценивания:**

- Задание выполнено верно, учтены все условия (**1 балл**): 
    * верно составлена сводная таблица итоговых результатов построенных ранее моделей с указанными в задании столбцами;
    * приведены итоговые выводы по проделанной работе, указано, какая из рассмотренных моделей показала наилучший результат.

- Задание выполнено неверно, не учтено одно или несколько условий (**0 баллов**).

In [ ]:
MODEL_NAMES = [
        "Линейная регрессия",
        "Lasso (L1 регуляризация)",
        "Ridge (L2 регуляризация)",
        "ElasticNet (L1 и L2 регуляризация)"
]
PARAMS = [
        "", 
        f"alpha=1",  # лассо
        f"alpha=0.596",  # ридж
        f"alpha=6.25, l1_ratio=1"
]
MSE_TRAIN = [
    10162,
    10403,  # лассо
    10309,  # ридж
    11775
]
MSE_TEST = [
    24335,
    20985,   # лассо
    23807,  # ридж
    17179  # эффект налицо!
]
MAE_TRAIN = [
    75,
    75,
    76,
    79  # казалось бы, на тренировке хуже всех
]
MAE_TEST = [
    110,
    102,
    110, # ридж тут плох
    95  #  а вот на валидации - лучше всех!
]

data = {
    "Наименование модели": MODEL_NAMES,
    "Гиперпараметры": PARAMS,
    "Полиномиальные признаки": [True, True, True, True], # мы ж везде добавили признаки третьего порядка
    
    "MSE на тренировочных фолдах": MSE_TRAIN,
    "MSE на валидационных фолдах": MSE_TEST,
    "MAE на тренировочных фолдах": MAE_TRAIN,
    "MAE на валидационных фолдах": MAE_TEST
}

results_df = pd.DataFrame(data)

results_df

,Наименование модели,Гиперпараметры,Полиномиальные признаки,MSE на тренировочных фолдах,MSE на валидационных фолдах,MAE на тренировочных фолдах,MAE на валидационных фолдах
0,Линейная регрессия,,True,10162,24335,75,110
1,Lasso (L1 регуляризация),alpha=1,True,10403,20985,75,102
2,Ridge (L2 регуляризация),alpha=0.596,True,10309,23807,76,110
3,ElasticNet (L1 и L2 регуляризация),"alpha=6.25, l1_ratio=1",True,11775,17179,79,95


Вот так круто себя показывает `ElasticNet`! На тренировке вроде и не лучший, а на валидации - самые низкие ошибки. Комбинация регуляризаций творит чудеса, и меньше переобучается 😉